In [ ]:
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

## 独自アルゴリズムを実装したコンテナの構築手順
1. container/modelフォルダのtrainにモデルの学習アルゴリズムを記述する<br>
サンプルではscikit-learnのロジスティック回帰を使用している<br>

2. container/modelフォルダのpredictor.py に推論アルゴリズムを記述する<br>
サンプルでは学習で作成したpickleを読み込み予測した結果を返すようにしている。<br>
pickleの読み込みなどが変わらなければ変更しなくてOK<br>

3. Dockerファイルの編集<br>
利用しているライブラリなどコンテナに追加で入れる必要があるものは追記する<br>

4. docker のコンテナをビルドしてECRに登録する<br>
1.〜3.の手順が終わったら以下のコマンドを実行してコンテナをECRに登録する<br>

In [ ]:
%%sh
cd container/
sh build_and_push.sh sagemaker-custom-algorithm-logistic

In [ ]:
# セッションの作成
import sagemaker
from time import gmtime, strftime

sess = sagemaker.Session()

In [ ]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
bucket = 'tokai.cognirobo.com'
bucket_path = 'https://s3-{}.amazonaws.com/{}'.format(region,bucket)

#print(account)
#print(region)
#print(bucket_path)

In [ ]:
from sagemaker.session import s3_input
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-custom-algorithm-logistic:latest'.format(account, region)

clf = sagemaker.estimator.Estimator(image,
                                    role,
                                    train_instance_count=1,
                                    train_instance_type='ml.m4.xlarge',
                                    output_path=bucket_path+'/output',
                                    sagemaker_session=sess
                                    )
#clf.set_hyperparameters(C=10.0)

s3_input_train = s3_input(s3_data=bucket_path + "/input/sagemaker_train.csv", content_type='text/csv')
s3_input_validation = s3_input(s3_data=bucket_path + "/input/sagemaker_valid.csv", content_type='text/csv')

In [ ]:
# 学習の実行
clf.fit({'training':s3_input_train, 'validation': s3_input_validation})

In [ ]:
from sagemaker.predictor import csv_serializer
predictor = clf.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer)

In [ ]:
shape=pd.read_csv("data/iris.csv", header=None)

import itertools

a = [50*i for i in range(3)]
b = [40+i for i in range(10)]
indices = [i+j for i,j in itertools.product(a,b)]

test_data=shape.iloc[indices[:-1]]

In [ ]:
print(predictor.predict(test_data.values).decode('utf-8'))

In [ ]:
sess.delete_endpoint(predictor.endpoint)

## Run Batch Transform Job

In [ ]:
transform_output_folder = "batch-transform-output"
output_path="s3://{}/{}".format(sess.default_bucket(), transform_output_folder)

transformer = clf.transformer(instance_count=1,
                               instance_type='ml.m4.xlarge',
                               output_path=output_path)

In [ ]:
transformer.transform(data_location, content_type='text/csv', split_type='Line')
transformer.wait()

In [ ]:
s3_client = sess.boto_session.client('s3')
s3_client.download_file(sess.default_bucket(), "{}/iris.csv.out".format(transform_output_folder), '/tmp/iris.csv.out')
with open('/tmp/iris.csv.out') as f:
    results = f.readlines()   
print("Transform results: \n{}".format(''.join(results)))